Query Transformations using Multi-Query 

In [1]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

True

In [2]:
# Access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

## Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')


os.environ['USER_AGENT'] = 'myagent'

In [3]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)

d:\work5\TRAGs\rgvenv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pprint import pprint
import bs4

In [5]:
from pinecone import ServerlessSpec

cloud = 'aws'
region = 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-3-small
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)

In [6]:
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [10]:
#### INDEXING ####
# Load Document (Uploading one file at a time)
pdf_file_path = "data/langchain_turing.pdf"
loader = PyPDFLoader(pdf_file_path)

In [11]:
docs = loader.load()

In [12]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, 
    chunk_overlap=500)

# Make splits
splits = text_splitter.split_documents(docs)

In [13]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [14]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(splits))]
uuids[0:3]

['03e039aa-b279-430e-9c24-fe3d4eddeaa8',
 'eaf362d8-35c2-46d3-a437-4fce325896d7',
 '734be04e-b30d-4ad3-b412-6208da6839a6']

In [15]:
vector_store.add_documents(documents=splits, ids=uuids)

['03e039aa-b279-430e-9c24-fe3d4eddeaa8',
 'eaf362d8-35c2-46d3-a437-4fce325896d7',
 '734be04e-b30d-4ad3-b412-6208da6839a6',
 '7261c8d9-d5fa-47cf-b1f3-d949a1def363',
 '5632c2cd-0209-4707-834f-7b1c1caf64dd',
 'cb4f5077-c928-4486-b220-f0d17339f1f5',
 '4268a529-9938-4608-b9e2-05829896cd09',
 '124225e8-95aa-44eb-ba1d-0befd3b66cab',
 '451a2f55-cfee-452a-8d8d-cc40e1ae149f',
 'f559762e-91e1-4973-a9d2-3286ed8730f2',
 'ff3b57ef-f8c6-4c6a-b50c-907fec0c527e',
 'd80c4c86-aca9-4cd4-a0c0-76a42fc77133',
 'b3199235-a7dd-4006-840e-2d53e321c23c',
 'a4f8004f-0fe7-4602-8835-820f49d77cdb']

In [21]:
retriever = vector_store.as_retriever()

In [17]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

In [18]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [19]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [22]:
# Retrieve
question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

C:\Users\shiv1\AppData\Local\Temp\ipykernel_15716\170002541.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


6

In [24]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"question":question}))

LangChain leverages modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications by providing tools for stateful process modeling, API deployment, monitoring, evaluation, and performance testing. LangGraph enables developers to structure applications with nodes and edges, allowing for complex branching and multi-agent workflows. LangServe facilitates the deployment of LLM applications as REST APIs, supporting scalability in production. LangSmith offers tools for real-time performance monitoring, error tracking, and version control to optimize applications iteratively. By integrating these components seamlessly and offering features like tracing, performance testing, API deployment, and scalability, LangChain provides developers with a comprehensive toolkit to build innovative and secure LLM-powered applications tailored to specific needs across diverse domains.
